In [1]:
import numpy as np
from matplotlib import pyplot as plt
import math as m

In [2]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display


def step_slice(lst, step):
    return lst[step]


def animate_list(lst, play=False, interval=200):
    slider = widgets.IntSlider(min=0, max=len(lst) - 1, step=1, value=0)
    if play:
        play_widjet = widgets.Play(interval=interval, max=len(lst) - 1)
        widgets.jslink((play_widjet, 'value'), (slider, 'value'))
        display(play_widjet)
    return interact(step_slice,
                    lst=fixed(lst),
                    step=slider)

In [3]:
#u_t = u_xx + 5*(1+cos(x+t)-sin(x+t))/(4*(1+cos(x+t))^2)
#u_x(0,t)=5/(4*(1+cos(t))
#u(1,t) + u_x(1,t) = 5*(1+sin(t+1))/(4*(1+cos(t+1)))
#u(x,0) = 5*tg(x/2)/4
#u_0(x,t) = 5*tg((x+t)/2)/4

In [4]:
a = 1.0

a1 = 1.0
b1 = 0

a2 = 1.0
b2 = 1.0


def f(x, t):
    return 5*(1+np.cos(x+t)-np.sin(x+t))/(4*(1+np.cos(x+t))**2)


def phi(x):
    return 5*np.tan(x/2)/4


def gamma1(t):
    return 5/(4*(1+np.cos(t)))


def gamma2(t):
    return 5*(1+np.sin(t+1))/(4*(1+np.cos(t+1)))


def exact(x, t):
    return 5*np.tan((x+t)/2)/4


x_min =  0
x_max = 1
h_cur_1 = 0.01

t_min = 0
t_max = 1
t_cur_1 = 0.01
              
h_cur_2 = h_cur_1 * 5
t_cur_2 = t_cur_1 * 5
              
sigma = 0.5

In [5]:
def thomas_algorithm(a,b,c,f,n):
    A = np.zeros(n)
    B = np.zeros(n)
    y = np.zeros(n)

    A[0] = -c[0] / b[0]
    B[0] = f[0] / b[0]

    for i in range(1, n - 1):
        A[i] = -c[i] / (b[i] + a[i] * A[i - 1])
    A[-1] = 0
    for i in range(1, n):
        B[i] = (f[i] - a[i] * B[i - 1]) / (b[i] + a[i] * A[i - 1])

    y[-1] = B[-1]
    for i in reversed(range(n-1)):
        y[i] = B[i] + A[i] * y[i + 1]
    return y

In [6]:
def aprox_1(step,h,t,gamma1,gamma2,U,a1,b1,a2,b2,a_,sigma):
    
    a = [0] + [t * (a_**2) * sigma / h**2 for i in range(1, len(U)-1)] + [-a2/h]
    
    b = [-(a1/h) + b1] + [-1 - 2 * t * (a_**2) * sigma/ h**2 for i in range(1, len(U)-1)] + [(a2 / h) + b2]

    c = [a1/h] + [ t * (a_**2) * sigma / h**2 for i in range(1, len(U)-1)] + [0]

    d = [- U[i] -t * f(h*i, (step - 0.5)*t) +(sigma-1)*(t * a_**2 / h ** 2)*(U[i+1] - 2*U[i] + U[i-1])
        for i in range(1, len(U)-1)]

    d = [gamma1(step*t)] + d + [gamma2(step*t)]
    
    return thomas_algorithm(a,b ,c ,d ,len(U))

In [9]:
def aprox_2(step,h,t,gamma1,gamma2,U,a1,b1,a2,b2,a_,sigma,x_min,x_max):

    a = [t * a_** 2 * sigma / (h ** 2) for i in range(1, len(U)-1)]
    b = [-1 - 2 * t * a_** 2 * sigma / (h ** 2) for i in range(1, len(U)-1)]
    c = [t * (a_** 2) * sigma / h ** 2 for i in range(1, len(U)-1)]
    d = [- U[i] - t * f(h*i, (step-0.5)*t) + (sigma-1) * (t* (a_**2)/(h**2)) * (U[i + 1] - 2 * U[i] + U[i - 1])
         for i in range(1, len(U)-1)]
    
    a = [0] + a
    
    if (a1 == 0):
        b = [b1] + b
        c = [0] + c
        d = [gamma1(step*t)] + d
    else:
        b = [1 - a_**2 * t / (h ** 2 * 2) * (-2 + b1 * 2 * h / a1)] + b
        c = [- a_**2 * t / (h ** 2)] + c
        d = [U[0]+a_**2*t/(2*h**2)*(-gamma1(step*t)*2*h/a1 + U[1] - 2*U[0]+U[1] - (gamma1((step-1)*t)-b1*U[0])*2*h/a1) + t*f(x_min, (step-0.5)*t)] + d

    c = c + [0]
    
    if (a2 == 0):
        a = a + [0]
        b = b + [b2]
        d = d + [gamma2(step*t)]
    else:
        a = a + [-a_**2*t/h**2]
        b = b + [1 - a_**2*t/(h**2*2)*(-2 - b2*2*h/a2)]
        d = d + [U[0]+a_**2*t/(2*h**2)*(gamma2(step*t)*2*h/a2 + U[-2] - 2*U[-1]+U[-2] + (gamma2((step-1)*t)-b2*U[0])*2*h/a2) + t*f(x_max, (step-0.5)*t)]

    return thomas_algorithm(a, b, c, d, len(U))

In [16]:
xk_cur_1 = np.arange(x_min, x_max + h_cur_1, h_cur_1)
tn_cur_1 = np.arange(t_min, t_max + t_cur_1, t_cur_1)

xk_cur_2 = np.arange(x_min, x_max + h_cur_2, h_cur_2)
tn_cur_2 = np.arange(t_min, t_max + t_cur_2, t_cur_2)

y1_ = [phi(i) for i in xk_cur_1]
ypol1 = []

y1__ = [phi(i) for i in xk_cur_1]
ypol1_ = []

y2_ = [phi(i) for i in xk_cur_2]
ypol2 = []

y2__ = [phi(i) for i in xk_cur_2]
ypol2_ = []


plots= []

err1 =[m.sqrt(np.mean(np.square(exact(xk_cur_1,0) - y1_)))]
err2 = [m.sqrt(np.mean(np.square(exact(xk_cur_2,0) - y2_)))]

err1_ =[m.sqrt(np.mean(np.square(exact(xk_cur_1,0) - y1__)))]
err2_ = [m.sqrt(np.mean(np.square(exact(xk_cur_2,0) - y2__)))]

err1_max =[max(exact(xk_cur_1,0) - y1_)]
err2_max = [max(exact(xk_cur_2,0) - y2_)]

err1_max_ =[max(exact(xk_cur_1,0) - y1__)]
err2_max_ = [max(exact(xk_cur_2,0) - y2__)]

fig, ax = plt.subplots(3, 1, figsize=(15, 15))

ax[0].plot(xk_cur_2, y2_, 'r', label="h = 5*10^-2, 1 порядок")
ax[0].plot(xk_cur_1, y1_, 'b', label="h = 10^-2, 1 порядок")

ax[0].plot(xk_cur_2, y2__, '--r', label="h = 5*10^-2, 2 порядок")
ax[0].plot(xk_cur_1, y1__, '--b', label="h = 10^-2, 2 порядок")

ax[0].plot(xk_cur_1, exact(xk_cur_1,0), 'g', label="exact")
ax[0].legend()
ax[0].set_xlabel('xk')
ax[0].set_ylabel('u')
ax[0].set_title('0.000')


ax[1].plot([0], err2, 'r', label="h = 5*10^-2, mse, 1 порядок")
ax[1].plot([0], err1, 'b', label="h = 10^-2, mse, 1 порядок")

ax[2].plot([0], err2_, '--r', label="h = 5*10^-2, mse, 2 порядок")
ax[2].plot([0], err1_, '--b', label="h = 10^-2, mse, 2 порядок")

ax[1].plot([0], err2_max, 'm', label="h = 5*10^-2, max, 1 порядок")
ax[1].plot([0], err1_max, 'c', label="h = 10^-2, max, 1 порядок")

ax[2].plot([0], err2_max_, '--m', label="h = 5*10^-2, max, 2 порядок")
ax[2].plot([0], err1_max_, '--c', label="h = 10^-2, max, 2 порядок")

ax[1].legend()
ax[1].set_title('Погрешности')
ax[1].set_xlabel('Время, t')

ax[2].legend()
ax[2].set_title('Погрешности')
ax[2].set_xlabel('Время, t')



plt.close(fig)
plots.append(fig)

for step in range(1, len(tn_cur_1)):
    

    ypol1 = aprox_1(step,h_cur_1,t_cur_1,gamma1,gamma2,y1_,a1,b1,a2,b2,a,sigma) #1 порядок h = 10^-2
    y1_ = ypol1.copy()
    
    ypol1_ = aprox_2(step,h_cur_1,t_cur_1,gamma1,gamma2,y1__,a1,b1,a2,b2,a,sigma,x_min,x_max) #2 порядок h = 10^-2
    y1__ = ypol1_.copy()
    
    
    if (step %5 == 0):

        ypol2 = aprox_1(step/5,h_cur_2,t_cur_2,gamma1,gamma2,y2_,a1,b1,a2,b2,a,sigma) #1 порядок h = 5*10^-2
        y2_ = ypol2.copy()
        
        ypol2_ = aprox_2(step/5,h_cur_2,t_cur_2,gamma1,gamma2,y2__,a1,b1,a2,b2,a,sigma,x_min,x_max) #2 порядок h = 5*10^-2
        y2__ = ypol2_.copy()
        
        err2_max.append(np.max(np.abs(exact(xk_cur_2,t_cur_2*step/5) - y2_)))
        err1_max.append(np.max(np.abs(exact(xk_cur_1,t_cur_1*step) - y1_)))
        
        err2_max_.append(np.max(np.abs(exact(xk_cur_2,t_cur_2*step/5) - y2__)))
        err1_max_.append(np.max(np.abs(exact(xk_cur_1,t_cur_1*step) - y1__)))
        
        err2.append(m.sqrt(np.mean(np.square(exact(xk_cur_2,t_cur_2*step/5) - y2_))))
        err1.append(m.sqrt(np.mean(np.square(exact(xk_cur_1,t_cur_1*step) - y1_))))
        
        err2_.append(m.sqrt(np.mean(np.square(exact(xk_cur_2,t_cur_2*step/5) - y2__))))
        err1_.append(m.sqrt(np.mean(np.square(exact(xk_cur_1,t_cur_1*step) - y1__))))
        
        fig, ax = plt.subplots(3, 1, figsize=(15, 15))

        ax[0].plot(xk_cur_2, y2_, 'r', label="h = 5*10^-2, 1 порядок")
        ax[0].plot(xk_cur_1, y1_, 'b', label="h = 10^-2, 1 порядок")

        ax[0].plot(xk_cur_2, y2__, '--r', label="h = 5*10^-2, 2 порядок")
        ax[0].plot(xk_cur_1, y1__, '--b', label="h = 10^-2, 2 порядок")
        
        ax[0].plot(xk_cur_1, exact(xk_cur_1,t_cur_1*step), 'g', label="exact")
        ax[0].legend()
        ax[0].set_xlabel('xk')
        ax[0].set_ylabel('u')
        ax[0].set_title('{:.2f}'.format(step*t_cur_1))

        

        ax[1].plot([i*t_cur_2 for i in range(len(err2))], err2, 'r', label="h = 5*10^-2, mse, 1 порядок")
        ax[1].plot([i*t_cur_2 for i in range(len(err1))], err1, 'b', label="h = 10^-3, mse, 1 порядок")
        
        ax[2].plot([i*t_cur_2 for i in range(len(err2_))], err2_, '--r', label="h = 5*10^-2, mse, 2 порядок")
        ax[2].plot([i*t_cur_2 for i in range(len(err1_))], err1_, '--b', label="h = 10^-3, mse, 2 порядок")
        
        ax[1].plot([i*t_cur_2 for i in range(len(err2_max))], err2_max, 'm', label="h = 5*10^-2, max, 1 порядок")
        ax[1].plot([i*t_cur_2 for i in range(len(err1_max))], err1_max, 'c', label="h = 10^-3, max, 1 порядок")
        
        ax[2].plot([i*t_cur_2 for i in range(len(err2_max_))], err2_max_, '--m', label="h = 5*10^-2, max, 2 порядок")
        ax[2].plot([i*t_cur_2 for i in range(len(err1_max_))], err1_max_, '--c', label="h = 10^-3, max, 2 порядок")
        
        ax[1].legend()
        ax[1].set_title('Погрешности')
        ax[1].set_xlabel('Время, t')
        
        ax[2].legend()
        ax[2].set_title('Погрешности')
        ax[2].set_xlabel('Время, t')

        
        plt.close(fig)
        plots.append(fig)
print('отношение max погрешностей mse, 1 порядок:', max(err2)/max(err1))
print('отношение max погрешностей max, 1 порядок:', max(err2_max)/max(err1_max))
print('отношение max погрешностей mse, 2 порядок:', max(err2_)/max(err1_))
print('отношение max погрешностей max, 2 порядок:', max(err2_max_)/max(err1_max_))

отношение max погрешностей mse, 1 порядок: 5.229753595587621
отношение max погрешностей max, 1 порядок: 5.080629636498819
отношение max погрешностей mse, 2 порядок: 25.24477521053739
отношение max погрешностей max, 2 порядок: 24.955201515802326


In [17]:
animate_list(plots, play=True, interval=250)

Play(value=0, interval=250, max=20)

interactive(children=(IntSlider(value=0, description='step', max=20), Output()), _dom_classes=('widget-interac…

<function __main__.step_slice(lst, step)>